In [ ]:
import cv2
import numpy as np
import imutils
from collections import deque

In [ ]:
#after testing hsv 

greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)
# greenLower = (41, 30, 74)
# greenUpper = (86, 144, 255)
pts = deque(maxlen=20)

In [ ]:
video = cv2.VideoCapture("test.mov")

fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
out = cv2.VideoWriter('outputtt.avi',fourcc, 25.0,(1000,562),True)

while(video.isOpened()):
    _, frame = video.read()
    if frame is None:
        break
    frame = imutils.resize(frame, width=1000)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
#     cv2.imshow("blurred",blurred)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
#     cv2.imshow("hsv",hsv)
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    kernel=np.ones((5,5),np.uint8)
#     mask = cv2.erode(mask, kernel, iterations=2)
#     mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
    mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    mask = cv2.dilate(mask, kernel, iterations=2)
    mask = cv2.erode(mask, kernel, iterations=2)
    res=cv2.bitwise_and(frame,frame,mask=mask)
    cnts,_=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]
#     cv2.imshow("mask",mask)
    
#     cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
    center = None
    
    # only proceed if at least one contour (ball) was found
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        center = (int(x), int(y))

        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)
    
    pts.appendleft(center)
    for i in range (1,len(pts)):
        if pts[i-1]is None or pts[i] is None:
            continue
        thick = int(np.sqrt(len(pts) / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i-1],pts[i],(0,0,225),thick)


#     frame = imutils.rotate(frame, 180)
    out.write(frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        break
        
        
video.release()
out.release()
cv2.destroyAllWindows()